In [1]:
import CalculatedFieldSubroutines as cfs

#

import numpy as np

import pandas as pd

#

import matplotlib.pyplot as plt

from pandasgui import show

#

import warnings

In [2]:
warnings.filterwarnings( 'ignore' )

In [3]:
gmID_list = cfs.list_whitelisted_gmIDs()

topic_list = cfs.list_topics()

print( topic_list )

['/apollo/canbus/chassis', '/apollo/drive/event', '/apollo/sensor/gnss/best/pose', '/apollo/perception/traffic/light']


In [8]:
cfs.retrieve_gmID_topic( gmID_list[ 0 ], '/apollo/canbus/chassis' ).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46118 entries, 0 to 46117
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   brakePercentage     46118 non-null  float64
 1   groupMetadataID     46118 non-null  object 
 2   drivingMode         46118 non-null  object 
 3   speedMps            46118 non-null  float64
 4   time                46118 non-null  int64  
 5   throttlePercentage  46118 non-null  float64
 6   steeringPercentage  46118 non-null  float64
 7   signal.turnSignal   46118 non-null  object 
dtypes: float64(4), int64(1), object(3)
memory usage: 2.8+ MB


In [ ]:
def RunningFunction( df, desired_colnames, operation, running_colname, window ):

    if ( operation == 'mean' ):

        def operation_func( arraylike_obj ): return np.mean( arraylike_obj )

    elif ( operation == 'median' ):

        def operation_func( arraylike_obj ): return np.median( arraylike_obj )

    elif ( operation == 'diff' ):

        def operation_func( arraylike_obj ): return ( arraylike_obj[ -1 ] - arraylike_obj[ 0 ] )

    else:

        raise Exception( "'operation' argument only supports: 'mean', 'median', 'diff'" )

    #

    running_col = np.array( df[ running_colname ] )

    running_col_diff = np.diff( running_col )

    #
    
    desired_cols = [ np.array( df[ desired_colname ] ) for desired_colname in desired_colnames ]

    #

    output_colnames = [ desired_colname + f'_{ operation }' for desired_colname in desired_colnames ]

    #

    window_indexes_list = []

    for index in range( len( running_col ) - 1 ):

        index = index + 1

        window_end_index = index - 1

        for window_start_index in range( window_end_index, -1 ):

            value_sum = np.sum( running_col_diff[ window_start_index : window_end_index + 1 ] )

            if ( value_sum >= window ):

                break

        window_indexes = [ index in range( window_start_index, window_end_index + 1 ) ]

        window_indexes_list.append( window_indexes )

    window_indexes_list = [ window_indexes_list[ 0 ] ] + window_indexes_list

    #

    

            
            

        

In [10]:
combined_dfs = []

for gmID in gmID_list:

    chassis_df = cfs.retrieve_gmID_topic( gmID, '/apollo/canbus/chassis' )

    best_pose_df = cfs.retrieve_gmID_topic( gmID, '/apollo/sensor/gnss/best/pose' )

    chassis_df = chassis_df.sort_values( 'time' )

    best_pose_df = best_pose_df.sort_values( 'time' )

    #

    cfs.Acceleration( chassis_df, time_interval = 1 / 3 )

    cfs.BinaryDrivingMode( chassis_df )

    cfs.TernaryDrivingModeTransition( chassis_df )

    cfs.DistanceToNearestDisengagement( chassis_df )

    #

    cfs.LatLonTotalStdDev( best_pose_df )

    cfs.ProgressAlongRoute_v2( best_pose_df )

    #

    cfs.ChassisBestPoseMatchedTime( chassis_df, best_pose_df )

    #

    combined_df = pd.merge( chassis_df, best_pose_df, on = 'ChassisBestPoseMatchedTime', how = 'inner' )

    #

    combined_df = combined_df[ [ 'groupMetadataID_x', 'brakePercentage', 'speedMps', 'time_x', 'throttlePercentage', 'steeringPercentage', \
                                 'TernaryDrivingModeTransition', 'DistanceToNearestDisengagement', 'NearestDisengagementID', \
                                 'longitude', 'latitude', 'LatLonTotalStdDev', 'ProgressAlongRoute', 'PartitionNumber', 'Acceleration' ] ]

    combined_dfs.append( combined_df )

In [11]:
combined_df_all = pd.concat( combined_dfs )

del combined_dfs

In [15]:
combined_df_all[ 'steeringPercentage' ] = np.abs( combined_df_all[ 'steeringPercentage' ] )

In [17]:
max_LatLonTotalStdDev = 10 #Free Variable 1

#

limited_stddev_list = []

for stddev in combined_df_all[ 'LatLonTotalStdDev' ]:

    if ( stddev > max_LatLonTotalStdDev ):

        stddev = max_LatLonTotalStdDev

    limited_stddev_list.append( stddev )

combined_df_all[ 'LatLonTotalStdDev' ] = limited_stddev_list

In [18]:
search_dist_behind_disengagement = 50 #m #Free Variable 1

combined_df_all_sliced = combined_df_all[ ( combined_df_all[ 'DistanceToNearestDisengagement' ] > -search_dist_behind_disengagement ) &
                                          ( combined_df_all[ 'DistanceToNearestDisengagement' ] < 0 ) ]